### Random Forest

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
df = pd.read_excel('./baseDeDados/dataBaseTratada.xlsx')

In [6]:
pd.set_option('display.max_columns', None)

df

,product_category_name,product_category_name_english,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,delivery_time,delivery_time_model,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,Latitude_Comprador,Longitude_Comprador,Latitude_Vendedor,Longitude_Vendedor,Distancia
0,utilidades_domesticas,housewares,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,sao paulo,e481f51cbdc54678b7cc49136f2d6af7,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.990000,8.720000,2,voucher,1,18.590000,a54f0611adc9ed256b57ede6b6eb5114,4.000000,Ausente,"""Não testei o produto ainda",NaT,NaT,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,8.440000,15.540000,40,268,4,500.0,19.0,8.0,13.0,9350,maua,SP,-23.5329,-46.6395,-23.6677,-46.4613,23.545096
1,utilidades_domesticas,housewares,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,sao paulo,e481f51cbdc54678b7cc49136f2d6af7,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.990000,8.720000,3,voucher,1,2.000000,a54f0611adc9ed256b57ede6b6eb5114,4.000000,Ausente,"""Não testei o produto ainda",NaT,NaT,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,8.440000,15.540000,40,268,4,500.0,19.0,8.0,13.0,9350,maua,SP,-23.5329,-46.6395,-23.6677,-46.4613,23.545096
2,utilidades_domesticas,housewares,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,sao paulo,e481f51cbdc54678b7cc49136f2d6af7,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.990000,8.720000,1,credit_card,1,169.157762,a54f0611adc9ed256b57ede6b6eb5114,4.000000,Ausente,"""Não testei o produto ainda",NaT,NaT,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,8.440000,15.540000,40,268,4,500.0,19.0,8.0,13.0,9350,maua,SP,-23.5329,-46.6395,-23.6677,-46.4613,23.545096
3,papelaria,stationery,8ab97904e6daea8866dbdbc4fb7aad2c,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,sao paulo,ad21c59c0840e6cb83a9ceb5573f8159,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,178.851413,8.720000,1,credit_card,1,28.620000,e50934924e227544ba8246aeb3770dd4,5.000000,Ausente,Ausente,2018-02-17,2018-02-18 13:02:51,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2.870000,25.244095,38,316,4,250.0,51.0,15.0,15.0,8752,mogi das cruzes,SP,-23.6737,-46.5432,-23.5208,-46.1854,40.228060
4,automotivo,auto,503740e9ca751ccdda7ba28e9ab8f608,80bb27c7c16e8f973207a5086ab329e2,86320,congonhinhas,parana,a4591c265e18cb1dcee52889e2d8acc3,1,060cb19345d90064d1015407193c233d,8581055ce74af1daba164fdbd55a40de,2017-07-13 22:10:13,147.900000,27.360000,1,credit_card,6,175.260000,89b738e70a1ce346db29a20fb2910161,4.000000,Ausente,Ausente,2017-07-27,2017-07-27 22:48:30,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-11 14:58:04,2017-07-26 10:57:55,16.540000,25.244095,49,608,1,7150.0,65.0,10.0,65.0,7112,guarulhos,SP,-23.4969,-50.6455,-23.4538,-46.5333,419.422496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73395,bebes,baby,1fca14ff2861355f6e5f14306ff977a7,da62f9e57a76d978d02ab5362c509660,11722,praia grande,sao paulo

In [61]:
# Selecionar as características que quero analisar e o alvo da previsão
features = ['customer_zip_code_prefix', 'price', 'freight_value', 'payment_value', 'payment_installments', 'payment_type_boleto', 'payment_type_credit_card', 'payment_type_debit_card', 'payment_type_voucher','product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'seller_zip_code_prefix', 'Distancia', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'dia_da_semana_Friday', 'dia_da_semana_Monday', 'dia_da_semana_Saturday', 'dia_da_semana_Sunday', 'dia_da_semana_Thursday', 'dia_da_semana_Tuesday', 'dia_da_semana_Wednesday', 'Latitude_Comprador', 'Longitude_Comprador', 'Latitude_Vendedor', 'Longitude_Vendedor']
target = 'delivery_time'

# Criando uma df para usar no modelo
df_corr = df[features].copy()
df_corr[target] = df[target].values

# Dividir dados para treino e para teste
X = df_corr.drop(columns=[target])
y = df_corr[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# Treinar o modelo Random Forest
model = RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=10, random_state=40)
model.fit(X_train, y_train)

# Fazer previsões e avaliar o modelo
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R²: {r2}')

Mean Squared Error: 49.16245384722604
R²: 0.22182685604304309


In [75]:
# Implementando validação cruzada

# Dividir os dados em X (features) e y (target)
X = df_corr.drop(columns=[target])
y = df_corr[target]

# Configurar a validação cruzada com KFold
kf = KFold(n_splits=4, shuffle=True, random_state=40)

# Listas para armazenar os resultados da validação cruzada
mse_list = []
r2_list = []
predictions_list = []

# Loop de validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Treinar o modelo Random Forest
    model = RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=10, random_state=40)
    model.fit(X_train, y_train)
    
    # Fazer previsões e avaliar o modelo
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Armazenar os resultados
    mse_list.append(mse)
    r2_list.append(r2)
    predictions_list.append(y_pred)

# Calcular a média e o desvio padrão dos resultados
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
mean_r2 = np.mean(r2_list)
std_r2 = np.std(r2_list)

# Combinar as previsões de todos os folds e calcular a média
all_predictions = np.concatenate(predictions_list)
mean_predictions = np.mean(all_predictions)

print(f'Mean Squared Error (Média ± DP): {mean_mse:.4f} ± {std_mse:.4f}')
print(f'R² (Média ± DP): {mean_r2:.4f} ± {std_r2:.4f}')
print(f'Média das previsões: {mean_predictions:.4f}')

Mean Squared Error (Média ± DP): 55.4145 ± 3.3934
R² (Média ± DP): 0.2138 ± 0.0085
Média das previsões: 13.2123


In [93]:
# Tentando encontrar modelo com melhores parâmetros

# Definir a grade de parâmetros
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Inicializar o modelo Random Forest
rf = RandomForestRegressor(random_state=40)

# Inicializar o GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# Ajustar o modelo aos dados
grid_search.fit(X_train, y_train)

# Melhor conjunto de parâmetros encontrados
best_params = grid_search.best_params_

print("Best parameters found: ", best_params)

# Treinar o modelo Random Forest com os melhores parâmetros
best_model = RandomForestRegressor(**best_params, random_state=40)
best_model.fit(X_train, y_train)

# Fazer previsões e avaliar o modelo
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R²: {r2}')

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Best parameters found:  {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error: 44.439905985419344
R²: 0.32930751849253437


In [62]:
# Importando a base do Kaggle
dfTeste = pd.read_excel('./baseDeDados/dataBaseTeste.xlsx')

In [63]:
dfTeste

,order_id,customer_id,order_purchase_timestamp,order_approved_at,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,Latitude_Comprador,Longitude_Comprador,Latitude_Vendedor,Longitude_Vendedor,Distancia,estado_vendedor_SP,estado_vendedor_MG,estado_vendedor_RJ,estado_comprador_sao paulo,estado_comprador_rio de janeiro,estado_comprador_minas gerais
0,cee3292f46ede6ea1dfabfcb200fcf47,0f8feee05f830b2be5955cf674a7d8f1,2018-06-21 15:00:00,2018-06-21 17:18:00,04d1643d1bf6c54f0b099c708e623466,8280,são paulo,sao paulo,1,1bfb290d7273a442c874dbe74b4abae6,7c67e1448b00f6e969d365cea6b010ab,2018-07-05 17:18:41,69.99,23.20,1,boleto,1,186.38,moveis_escritorio,19.0,883.0,1.0,10075,55,64,26,8577,itaquaquecetuba,SP,-23.5329,-46.6395,-23.4835,-46.3457,30.457066,1,0,0,1,0,0
1,50dca53ca33b739bef09e7933e8b380e,d0580aed7f303a707d93b00b84c591a7,2018-07-14 09:47:00,2018-07-14 10:04:00,83f6b0c43f9281884da6fb400b587207,7151,guarulhos,sao paulo,1,eb048e296b8db6cd64b1bafd9500786c,1900267e848ceeba8fa32d80c1a5f5a8,2018-07-17 10:04:14,19.90,12.80,1,credit_card,3,65.40,cama_mesa_banho,52.0,149.0,1.0,250,16,10,16,14940,ibitinga,SP,-23.4538,-46.5333,-22.0388,-48.3342,242.607747,1,0,0,1,0,0
2,8087ec71e393d4dc6fc48041fe63cd51,b8dfb418e6cad762b91febf827d4b2e5,2018-05-14 10:03:00,2018-05-14 10:15:00,ae85516c85361785d9151930418af0cb,4013,sao paulo,sao paulo,1,39cdb64dad488a6037d828d1cd514255,8f119a0aee85c0c8fc534629734e94fd,2018-05-18 10:15:20,22.50,8.64,1,credit_card,3,179.68,casa_construcao,60.0,890.0,2.0,300,28,9,18,6440,barueri,SP,-23.5329,-46.6395,-23.5057,-46.8790,24.605409,1,0,0,1,0,0
3,e6b6557ce111de79b31cc857f20ba212,90c8fe0d707ea62be3fcb5abd7bc784f,2018-05-22 18:41:00,2018-05-22 18:57:00,5b1ce2603aed92f3655f96818402e8ec,30660,belo horizonte,minas gerais,1,803f77475e1b51b47f1bfec4f2ec353f,c9c7905cffc4ef9ff9f113554423e671,2018-05-25 18:57:37,69.99,15.37,1,credit_card,1,85.36,automotivo,58.0,672.0,3.0,350,17,6,20,6871,itapecerica da serra,SP,-19.9102,-43.9266,-23.7161,-46.8572,520.164285,1,0,0,0,0,1
4,0b09c5e4c2512f627190ac55a78c35a3,6ba83000cf8c24413916a7a72fcb5173,2018-04-30 19:31:00,2018-05-03 04:12:00,ae3b3ea754f62eaa12630c4e6f69750f,89056,blumenau,santa catarina,1,89321f94e35fc6d7903d36f74e351d40,16090f2ca825584b5a147ab24aa30c86,2018-05-09 04:12:51,27.90,18.23,1,boleto,1,46.13,alimentos,59.0,982.0,1.0,150,17,13,13,12940,atibaia,SP,-26.9155,-49.0709,-27.4867,-50.9724,198.487101,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28147,81cedc4d2132eb6f512354166e51b187,a8f6bb35d78515a1762eb626d52b6352,2018-07-30 21:02:00,2018-07-30 21:10:00,d0ad8d078320fd639fe1ef1bcbe0e4af,12705,cruzeiro,sao paulo,1,0d0bc6cf9765ffe8b4d3103138c96cdd,179d859f51292aac5c10806a356f7af9,2018-08-05 21:10:20,17.49,12.79,1,credit_card,1,30.28,eletronicos,60.0,436.0,1.0,200,19,13,17,9411,ribeirao pires,SP,-22.5728,-44.9690,-23.7067,-46.4058,193.594736,1,0,0,1,0,0
28148,95811675a89b66659996da5285a021db,5885969ba920706beb8d5381a2ba981c,2018-08-09 11:07:00,2018-08-09 15:25:00,d714959800e10629872c11353cbb1913,72002,brasília,distrito federal,1,aca2eb7d00ea1a7b8ebd4e68314663af,955fee9216a65b617aa5c0531780ce60,2018-08-13 15:25:22,109.90,22.07,1,credit_card,9,131.97,moveis_decoracao,44.0,903.0,6.0,2600,50,10,30,4782,sao paulo,SP,-15.7795,-47.9297,-23.5329,-46.6395,872.638307,1,0,0,0,0,0
28149,25bd2ad9d282f59e4e6a048ef9bf214f,be34bf56044c5614fc66d97251c0adc6,2018-03-12 16:20:00,2018-03-12 17:09:00,768e9ed92dd4fe880672c8e90dc4f74c,66610,belem,para,1,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f90235231

In [64]:
# Extrair o dia da semana
dfTeste['dia_da_semana'] = dfTeste['order_approved_at'].dt.day_name()

# Criar colunas binárias para cada dia da semana
dfTeste = pd.get_dummies(dfTeste, columns=['dia_da_semana'])

# Converter colunas binárias para 1 e 0
for col in dfTeste.columns:
    if 'dia_da_semana' in col:
        dfTeste[col] = dfTeste[col].astype(int)

In [65]:
# Converter a coluna para o formato category
dfTeste['payment_type'] = dfTeste['payment_type'].astype('category')

# Criar colunas binárias para cada tipo de pagamento
dfTeste = pd.get_dummies(dfTeste, columns=['payment_type'])

# Converter colunas binárias para 1 e 0
for col in dfTeste.columns:
    if 'payment_type' in col:
        dfTeste[col] = dfTeste[col].astype(int)

In [76]:
# Estabelecendo X test a partir das mesmas features
X_test_dfTeste = dfTeste[features]

# Realizando as previsões
dfTeste['previsoes'] = model.predict(X_test_dfTeste)

# Analisando os resultados
dfTeste[['order_id', 'previsoes']]

,order_id,previsoes
0,cee3292f46ede6ea1dfabfcb200fcf47,16.950478
1,50dca53ca33b739bef09e7933e8b380e,11.646903
2,8087ec71e393d4dc6fc48041fe63cd51,7.676388
3,e6b6557ce111de79b31cc857f20ba212,12.798570
4,0b09c5e4c2512f627190ac55a78c35a3,12.986027
...,...,...
28147,81cedc4d2132eb6f512354166e51b187,11.682067
28148,95811675a89b66659996da5285a021db,13.844416
28149,25bd2ad9d282f59e4e6a048ef9bf214f,21.761678
28150,9f9ac312604229686c04260955495c70,7.591500


In [72]:
# Previsões com validação cruzada

# Configurar a validação cruzada com KFold
kf = KFold(n_splits=5, shuffle=True, random_state=40)

# Armazenar previsões para cada linha no conjunto de teste
test_predictions = np.zeros((dfTeste.shape[0], kf.n_splits))

# Loop de validação cruzada
for fold_idx, (train_index, _) in enumerate(kf.split(X)):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    # Treinar o modelo Random Forest
    model = RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=10, random_state=40)
    model.fit(X_train, y_train)
    
    # Realizar previsões no conjunto de teste
    X_test_dfTeste = dfTeste[features]
    test_predictions[:, fold_idx] = model.predict(X_test_dfTeste)

# Calcular a média das previsões para cada pedido no conjunto de teste
dfTeste['previsoes'] = test_predictions.mean(axis=1)

# Analisando os resultados
dfTeste[['order_id', 'previsoes']]

,order_id,previsoes
0,cee3292f46ede6ea1dfabfcb200fcf47,16.143530
1,50dca53ca33b739bef09e7933e8b380e,11.743145
2,8087ec71e393d4dc6fc48041fe63cd51,7.506444
3,e6b6557ce111de79b31cc857f20ba212,12.841518
4,0b09c5e4c2512f627190ac55a78c35a3,13.246581
...,...,...
28147,81cedc4d2132eb6f512354166e51b187,11.482780
28148,95811675a89b66659996da5285a021db,13.847161
28149,25bd2ad9d282f59e4e6a048ef9bf214f,21.532658
28150,9f9ac312604229686c04260955495c70,8.130058


In [77]:
dfTeste[['order_id', 'previsoes']].describe()

,previsoes
count,28152.000000
mean,13.407094
std,3.700060
min,6.655238
25%,10.969030
50%,13.246546
75%,15.843364
max,47.152842


In [74]:
# Exportando para um csv
test_RandomForest_csv = dfTeste[['order_id', 'previsoes']]

test_RandomForest_csv.to_csv('./Previsoes/PrevisoesRandomForest.csv', index=False)